In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys

In [2]:
# GLOBALS
LOCAL_ROOT = "/Users/varunnathan/Documents/General"
PROJ_DIR = os.path.join(LOCAL_ROOT, "ExternalTest/Prodigal")
RAW_DATA_DIR = os.path.join(PROJ_DIR, "raw")
INTER_DATA_DIR = os.path.join(PROJ_DIR, "intermediate")
RESULTS_DIR = os.path.join(PROJ_DIR, "results")
MODEL_DIR = os.path.join(PROJ_DIR, "model")
TRAIN_FN = os.path.join(RAW_DATA_DIR, "final_train.json")

In [3]:
# read data
df = json.load(open(TRAIN_FN))

In [5]:
type(df), len(df)

(list, 50)

In [9]:
df[0]

{'_id': 'fd69a1f8-6bb0-4624-8052-b6e46fe7622a',
 'text': "nine seven three and then let's just go over with this arrangement and then we'll be completely set please state mister is december twenty first two thousand twenty authorize authorized one time transaction and of twelve dollar and ninety seven cents be address from your card will be processed as an electronic and solutions and your stevens if you just need to cause it's recording shipping is your electronic signature can please speak your full name ai i agree",
 'entity': [{'span': [123, 164],
   'string': 'december twenty first two thousand twenty',
   'type': 'DATE'},
  {'span': [186, 194], 'string': 'one time', 'type': 'NUM_PAYMENTS'},
  {'span': [214, 250],
   'string': 'twelve dollar and ninety seven cents',
   'type': 'PAYMENT_AMOUNT'}]}

In [30]:
cnt = 0
ids = set()
for row in df:
    ids.add(row['_id'])
    cnt += 1
print(len(ids), cnt)

50 50


In [34]:
df[0]['text'].find('december twenty first two thousand twenty')
123 + len('december twenty first two thousand twenty')

164

In [7]:
# support for each class
from collections import defaultdict

LABELS = ['DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_DATE']

label_support_dct = defaultdict(list)
for row in df:
    for ent in row['entity']:
        cur_cnt = label_support_dct.get(ent['type'], 0)
        label_support_dct[ent['type']] = cur_cnt + 1
    
for key, cnt in label_support_dct.items():
    print(key, '\t', cnt)

DATE 	 51
NUM_PAYMENTS 	 38
PAYMENT_AMOUNT 	 58
PAYMENT_DATE 	 46
PAYMEN_AMOUNT 	 1


In [13]:
# replace PAYMEN_AMOUNT with PAYMENT_AMOUNT
for i, row in enumerate(df):
    for j, ent in enumerate(row['entity']):
        if ent['type'] == 'PAYMEN_AMOUNT':
            df[i]['entity'][j]['type'] = 'PAYMENT_AMOUNT'
            
label_support_dct = defaultdict(list)
for row in df:
    for ent in row['entity']:
        cur_cnt = label_support_dct.get(ent['type'], 0)
        label_support_dct[ent['type']] = cur_cnt + 1
    
for key, cnt in label_support_dct.items():
    print(key, '\t', cnt)

DATE 	 51
NUM_PAYMENTS 	 38
PAYMENT_AMOUNT 	 59
PAYMENT_DATE 	 46


In [16]:
# find rows with all labels
for i, row in enumerate(df):
    labels = [ent['type'] for ent in row['entity']]
    if len(set(labels)) == len(LABELS):
        print(i, len(labels), labels)
        

3 8 ['DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_AMOUNT', 'PAYMENT_DATE', 'PAYMENT_DATE', 'PAYMENT_DATE', 'PAYMENT_AMOUNT']
9 6 ['NUM_PAYMENTS', 'PAYMENT_DATE', 'DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_AMOUNT']
10 4 ['DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_DATE']
14 5 ['DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_DATE', 'PAYMENT_DATE']
16 4 ['DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_DATE']
28 4 ['DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_DATE']
38 4 ['DATE', 'NUM_PAYMENTS', 'PAYMENT_DATE', 'PAYMENT_AMOUNT']
39 4 ['DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_DATE']
44 4 ['DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_DATE']
46 4 ['DATE', 'NUM_PAYMENTS', 'PAYMENT_AMOUNT', 'PAYMENT_DATE']


In [17]:
from sklearn.model_selection import LeaveOneOut

In [18]:
loo = LeaveOneOut()

In [19]:
loo.get_n_splits(df)

50

In [37]:
for train_index, test_index in loo.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = [x for i, x in enumerate(df) if i not in test_index], [df[test_index[0]]]
    print(len(X_train), len(X_test))
    break

TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49] TEST: [0]
49 1
